In [43]:
import json
import requests

response = requests.get("https://restcountries.com/v3.1/all")
countries = response.json()

with open("countries.json", "w") as json_file:
    json.dump(countries, json_file, indent=2)

In [44]:
def flatten_nested_structure(data, parent_key="", sep="."):
    items = {}
    if isinstance(data, dict):
        for k, v in data.items():
            new_key = f"{parent_key}{sep}{k}" if parent_key else k
            items.update(flatten_nested_structure(v, new_key, sep))
    elif isinstance(data, list):
        if all(not isinstance(i, (dict, list)) for i in data):
            # Join list items as strings, preserving emojis
            items[parent_key] = ", ".join(str(i) for i in data)
        else:
            for i, v in enumerate(data):
                new_key = f"{parent_key}{sep}{i}" if parent_key else str(i)
                items.update(flatten_nested_structure(v, new_key, sep))
    else:
        items[parent_key] = str(data)
    return items

In [45]:
flat_countries = []

for country in countries:
    flat_country = flatten_nested_structure(country)
    flat_countries.append(flat_country)

sorted_countries = sorted(
    flat_countries,
    key=lambda x: (x.get("region", ""), x.get("subregion", ""), x.get("name.common", "")),
)

with open("flat_countries.json", "w") as json_file:
    json.dump(sorted_countries, json_file, indent=2)

In [46]:
import pandas as pd

countries_df = pd.DataFrame(sorted_countries)
simplified_countries_df = countries_df[
    [
        "flag",
        "cca2",
        "cca3",
        "region",
        "subregion",
        "name.common",
        "name.official",
        "capital",
    ]
]
simplified_countries_df.fillna("", inplace=True)
simplified_countries = simplified_countries_df.to_dict(orient="records")
with open("simplified_countries.json", "w", encoding="utf-8") as f:
    json.dump(simplified_countries, f, ensure_ascii=False, indent=2)

/var/folders/jh/z981c7zj0vz0gmyfc8mhdxdr0000gn/T/ipykernel_6290/3350331896.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simplified_countries_df.fillna("", inplace=True)
